# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

    # Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# Uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
            
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
# Creation of a Keyspace called 'udacity'
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION =
        { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1: Get the artist name, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Primary Key: Since the sessionId and itemInSession columns are used with filter together with the WHERE clause, these columns would form our composite primary key.

In [7]:
# Create statement
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songs_session(
            session_id int,
            item_in_session int,
            artist_name text,
            song_name text,
            song_length float,
            PRIMARY KEY (session_id, item_in_session)
        )
    """)
except Exception as e:
    print(e)

In [8]:
# Set up the CSV file
file = 'event_datafile_new.csv'

# Open the CSV file provided
with open(file, encoding = 'utf8') as f:
    
    # Read our CSV file
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Insert statement for filling songs_session table
        query = "INSERT INTO songs_session (session_id, item_in_session,\
                artist_name, song_name, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        # Assign which column element and type should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into songs_session table

In [9]:
# SELECT statement to verify the data was entered into the table songs_session
query = """
    SELECT artist_name, song_name, song_length
    FROM songs_session
    WHERE session_id = 338 AND item_in_session = 4
"""

try:
    rows_result = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(rows_result._current_rows)
print(df)

  artist_name                        song_name  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2: Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Primary Key: Since the userid and sessionid columns are used with filter together with the WHERE clause, these columns would form our partition columns. In addition, the query tells us to obtain the songs ordered by the itemInSession column, which becomes our clustering column, having a primary key composed of our partition columns and clustering column respectively.

In [10]:
# Create statement
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS users_session(
            user_id int,
            session_id int,
            item_in_session int,
            artist_name text,
            song_name text,
            first_name text,
            last_name text,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        )
    """)
except Exception as e:
    print(e)

In [11]:
# Set up the CSV file
file = 'event_datafile_new.csv'

# Open the CSV file provided
with open(file, encoding = 'utf8') as f:
    
    # Read our CSV file
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Insert statement for filling users_session table
        query = "INSERT INTO users_session (user_id, session_id, item_in_session,\
                artist_name, song_name, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Assign which column element and type should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into users_session table

In [13]:
# SELECT statement to verify the data was entered into the table users_session
query = """
    SELECT artist_name, song_name, first_name, last_name
    FROM users_session
    WHERE user_id = 10 AND session_id = 182
"""

try:
    rows_result = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(rows_result._current_rows)
print(df)

         artist_name                                          song_name  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


## Query 3: Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Primary Key: Since the song_name column is used with filter, these column would form our partition column. In addition and uniquely for identifying unique rows, we add the user_id column together the partition column to create a composite primary key.

In [14]:
# Create statement
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songs_user(
            song_name text,
            user_id int,
            first_name text,
            last_name text,
            PRIMARY KEY (song_name, user_id)
        )
    """)
except Exception as e:
    print(e)

In [15]:
# Set up the CSV file
file = 'event_datafile_new.csv'

# Open the CSV file provided
with open(file, encoding = 'utf8') as f:
    
    # Read our CSV file
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Insert statement for filling songs_user table
        query = "INSERT INTO songs_user (song_name, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element and type should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into songs_user table

In [16]:
# SELECT statement to verify the data was entered into the table songs_user
query = """
    SELECT first_name, last_name
    FROM songs_user
    WHERE song_name='All Hands Against His Own'
"""

try:
    rows_result = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(rows_result._current_rows)
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [18]:
# Drop the tables before closing out the sessions
tables_arr = ['songs_session', 'users_session', 'songs_user']

for table in tables_arr:
    try:
        session.execute(f"DROP TABLE IF EXISTS {table}")
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()